In [1]:
import os
import time
import uuid
import cv2

In [2]:
IMAGES_PATH = 'data'
number_images = 10

In [3]:
cap = cv2.VideoCapture(0)
for imgnum in range(number_images):
    print('Collecting image {}'.format(imgnum))
    ret, frame = cap.read()
    imgname = os.path.join(IMAGES_PATH,f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(imgname, frame)
    cv2.imshow('frame', frame)
    time.sleep(0.5)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

### Create Dataset

In [4]:
import tensorflow as tf
import cv2
import json
import numpy as np
from matplotlib import pyplot as plt

In [5]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
def load_image(x): 
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

In [7]:
for folder in ['train','test','val']:
    for file in os.listdir(os.path.join('data', folder, 'images')):
        
        filename = file.split('.')[0]+'.json'
        existing_filepath = os.path.join('data','labels', filename)
        if os.path.exists(existing_filepath): 
            new_filepath = os.path.join('data',folder,'labels',filename)
            os.replace(existing_filepath, new_filepath) 

### Image Augmentation

In [8]:
import albumentations as alb

In [9]:
augmentor = alb.Compose([alb.RandomCrop(width=450, height=450), 
                         alb.HorizontalFlip(p=0.5), 
                         alb.RandomBrightnessContrast(p=0.2),
                         alb.RandomGamma(p=0.2), 
                         alb.RGBShift(p=0.2), 
                         alb.VerticalFlip(p=0.5)], 
                        keypoint_params=alb.KeypointParams(format='xy', label_fields=['class_labels']))

In [10]:
for partition in ['train', 'test', 'val']: 
    for image in os.listdir(os.path.join('data', partition, 'images')):
        img = cv2.imread(os.path.join('data', partition, 'images', image))

        classes = [0, 0]
        coords = [0, 0, 0, 0]  # Fix the initialization of coords
        label_path = os.path.join('data', partition, 'labels', f'{image.split(".")[0]}.json')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)

            for shape in label['shapes']:
                if shape['label'] == 'LeftEye':
                    classes[0] = 1
                    coords[0] = shape['points'][0][0]  # Fix the coordinate access
                    coords[1] = shape['points'][0][1]
                elif shape['label'] == 'RightEye':
                    classes[1] = 1
                    coords[2] = shape['points'][0][0]  # Fix the coordinate access
                    coords[3] = shape['points'][0][1]

            # Now you don't need to check for the existence of the second shape, as the loop handles all shapes
            np.divide(coords, [640, 480, 640, 480])

        try: 
            for x in range(120):
                keypoints = [(coords[:2]), (coords[2:])]
                augmented = augmentor(image=img, keypoints=keypoints, class_labels=['LeftEye', 'RightEye'])
                cv2.imwrite(os.path.join('data', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

                annotation = {
                    'image': image,
                    'class': classes,
                    'keypoints': coords
                }

                if os.path.exists(label_path) and len(augmented['keypoints']) > 0:
                    for idx, cl in enumerate(augmented['class_labels']):
                        if cl == 'LeftEye':
                            annotation['keypoints'][:2] = augmented['keypoints'][idx]
                        elif cl == 'RightEye':
                            annotation['keypoints'][2:] = augmented['keypoints'][idx]

                annotation['keypoints'] = list(np.divide(annotation['keypoints'], [450, 450, 450, 450]))

                with open(os.path.join('data', partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)